In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.cross_validation import train_test_split

/home/sergey/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/sergey/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
import numpy as np

In [3]:
def preprocess_image(img):
    
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [4]:
image_gen_train = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #zca_whitening=True,
    rescale=1./255,
    rotation_range = 90,
    width_shift_range=.5,
    height_shift_range=.5,
    shear_range=90,
    zoom_range= [3, 3],
    #channel_shift_range: Float. Range for random channel shifts.
    horizontal_flip=True,
    vertical_flip= True,
    preprocessing_function = preprocess_image
)
image_gen_val = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #zca_whitening=True,
    rescale=1./255,
    preprocessing_function = preprocess_image
)

#training the image preprocessing
#image_gen.fit(x_train, augment=True)

In [5]:
train_generator = image_gen_train.flow_from_directory(
        './input/train_resized_224/train',
        target_size=(224, 224),
        batch_size=32)

validation_generator = image_gen_val.flow_from_directory(
        './input/train_resized_224/val',
        target_size=(224, 224),
        batch_size=32)

Found 974865 images belonging to 14951 classes.
Found 242443 images belonging to 14951 classes.


In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import ImageFile
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard

In [6]:
def read_img(address, size):
    """Read and resize image.
    Returns Image as numpy array, by normalizing the values
    """
    img = image.load_img(address, target_size=size)
    img = image.img_to_array(img)
    return img


In [27]:
data = pd.read_csv('./input/train.csv', skiprows=1, nrows=3, names = ['id', 'source', 'landmark_id'])
data.head()

,id,source,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284


In [23]:
INPUT_SIZE = 224

#POOLING = 'avg'
def load_image(skip, nrows):
    data = pd.read_csv('./input/train.csv', skiprows=skip, nrows=nrows, names = ['id', 'source', 'landmark_id'])
    labels = data.shape[0]
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    x_train = np.zeros((labels, INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
    y_train= np.zeros((labels,), dtype='float32')
    for i, img_id in tqdm(enumerate(data['id'])):
        try:
            img = read_img('./input/train_resized_224/'+img_id+'.jpg', (INPUT_SIZE, INPUT_SIZE))
            x = preprocess_input(np.expand_dims(img.copy(), axis=0))
            x_train[i] = x
            y_train[i]=int(data['landmark_id'][i])
        except:
            continue 
    return x_train, y_train

In [8]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
data = pd.read_csv('./input/train.csv')
data = data.groupby(['landmark_id']).head(20)
labels = data.landmark_id.nunique()
enc.fit(data['landmark_id'].astype(int).values.reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [7]:
from keras.callbacks import EarlyStopping

es = EarlyStopping( 'val_loss', patience = 5, 
                        mode = 'min', min_delta = .1)

save_check = ModelCheckpoint('./model/model_step_1.h5', monitor='val_loss',
                verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, 
                                 write_graph=True, write_grads=False, write_images=False, 
                                 embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)


In [8]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(14951, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
from gc import collect
collect()

166065

In [13]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

model.fit_generator(train_generator,
                       verbose = 1,
                         validation_data=validation_generator, 
                      callbacks=[save_check, es, tb],
                        steps_per_epoch = 200, epochs=30)
    
    # train the model on the new data for a few epochs
    #model.fit_generator(...)
# serialize model to JSON
model_json = model.to_json()
with open("./model/model_step_1_end.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./model/model_step_1_end.h5")
print("Saved model to disk")

Epoch 1/30
200/200 [==============================] - 9833s 49s/step - loss: 8.3441 - val_loss: 8.5200

Epoch 00001: val_loss improved from inf to 8.52005, saving model to ./model/model_step_1.h5
Epoch 2/30
200/200 [==============================] - 1423s 7s/step - loss: 7.9744 - val_loss: 8.1336

Epoch 00002: val_loss improved from 8.52005 to 8.13357, saving model to ./model/model_step_1.h5
Epoch 3/30
200/200 [==============================] - 1286s 6s/step - loss: 7.9080 - val_loss: 8.2842

Epoch 00003: val_loss did not improve from 8.13357
Epoch 4/30
200/200 [==============================] - 1314s 7s/step - loss: 7.8590 - val_loss: 8.1075

Epoch 00004: val_loss improved from 8.13357 to 8.10752, saving model to ./model/model_step_1.h5
Epoch 5/30
200/200 [==============================] - 1310s 7s/step - loss: 7.8292 - val_loss: 8.0795

Epoch 00005: val_loss improved from 8.10752 to 8.07947, saving model to ./model/model_step_1.h5
Epoch 6/30
200/200 [==============================] -

In [10]:
model.load_weights('./model/model_step_1.h5')

In [11]:
save_check = ModelCheckpoint('./model/model_step_2.h5', monitor='val_loss',
                verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

In [12]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13560514697701658143
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5555093504
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12917793432077805929
physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
#for i, layer in enumerate(base_model.layers):
    #print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

model.fit_generator(train_generator,
                       verbose = 1,
                         validation_data=validation_generator, 
                      callbacks=[save_check, es, tb],
                        steps_per_epoch = 100, epochs=3)

Epoch 1/3
100/100 [==============================] - 3677s 37s/step - loss: 7.8019 - val_loss: 7.9109

Epoch 00001: val_loss improved from inf to 7.91091, saving model to ./model/model_step_2.h5
Epoch 2/3
100/100 [==============================] - 1243s 12s/step - loss: 7.7291 - val_loss: 7.8945

Epoch 00002: val_loss improved from 7.91091 to 7.89448, saving model to ./model/model_step_2.h5
Epoch 3/3
 99/100 [============================>.] - ETA: 0s - loss: 7.6421

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sergey/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-f8a20cc07094>", line 26, in <module>
    steps_per_epoch = 100, epochs=3)
  File "/home/sergey/.local/lib/python3.6/site-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/home/sergey/.local/lib/python3.6/site-packages/keras/engine/training.py", line 2250, in fit_generator
    max_queue_size=max_queue_size)
  File "/home/sergey/.local/lib/python3.6/site-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/home/sergey/.local/lib/python3.6/site-packages/keras/engine/training.py", line 2399, in evaluate_generator
    outs = self.test_on_batch(x, y, sample_weight=sample_weight)
  File "/home/sergey/.local/lib/python3.6/site-packages/keras/engine/training.py", line 192

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/sergey/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/home/sergey/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/sergey/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/sergey/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/sergey/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/home/sergey/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/home/sergey/anaconda3/lib/